# Objetivos deste trabalho
- Familiarizar-se com a biblioteca PyTorch
- Definir arquiteturas MLP simples em PyTorch
- Treinar utilizando CIFAR10, testando diferentes arquiteturas, parâmetros, funções de loss e otimizadores
- Comparar os resultados obtidos utilizando apenas Perpceptrons

## Inicialização 

In [14]:
%matplotlib inline

import time

import numpy as np 
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [15]:
# Carregar os datasets

transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

dataset_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

dataset_test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [16]:
train_loader = DataLoader(dataset=dataset_train, batch_size=50, 
                          shuffle=True, num_workers=2)
test_loader = DataLoader(dataset=dataset_test, batch_size=50, 
                         shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Definindo as Arquiteturas MLP

In [17]:
# Arquitetura MLP com uma Camada Oculta
class MLP1(nn.Module):
    
    # Construtor da Classe MLP
    def __init__(self):
        super(MLP1, self).__init__()
        self.fc1 = nn.Linear(32*32, 512)
        self.fc2 = nn.Linear(512, 10)
        self.activation_function = nn.ReLU()
    
    # Função Forward
    def forward(self, x):
        x = x.view(-1, 32*32)
        x = self.activation_function(self.fc1(x))
        x = self.activation_function(self.fc2(x))
        return x

# Arquitetura MLP com duas Camadas Ocultas
class MLP2(nn.Module):
    
    # Construtor da Classe MLP
    def __init__(self):
        super(MLP2, self).__init__()
        self.fc1 = nn.Linear(32*32, 512)
        self.fc2 = nn.Linear(512,   256)
        self.fc3 = nn.Linear(256,    10)
        self.activation_function = nn.ReLU()
    
    # Função Forward
    def forward(self, x):
        x = x.view(-1, 32*32)
        x = self.activation_function(self.fc1(x))
        x = self.activation_function(self.fc2(x))
        x = self.activation_function(self.fc3(x))
        return x

# Arquitetura MLP com três Camadas Ocultas
class MLP3(nn.Module):
    
    # Construtor da Classe MLP
    def __init__(self):
        super(MLP3, self).__init__()
        self.fc1 = nn.Linear(32*32, 512)
        self.fc2 = nn.Linear(512,   256)
        self.fc3 = nn.Linear(256,   128)
        self.fc4 = nn.Linear(128,    10)
        self.activation_function = nn.ReLU()
    
    # Função Forward
    def forward(self, x):
        x = x.view(-1, 32*32)
        x = self.activation_function(self.fc1(x))
        x = self.activation_function(self.fc2(x))
        x = self.activation_function(self.fc3(x))
        x = self.activation_function(self.fc4(x))
        return x

### Verificação se é possivel utilizar _cuda_

In [18]:
# Confere se pode utilizar cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Imprime o dispotivo
print(device)

cpu


### Impressão dos modelos

In [19]:
# MLP com 1 camada oculta
model1 = MLP1()
model1 = model1.to(device)

# MLP com 2 camadas ocultas
model2 = MLP2()
model2 = model2.to(device)

# MLP com 3 camadas ocultas
model3 = MLP3()
model3 = model3.to(device)

# Imprime os modelos
print(model1)
print(model2)
print(model3)

MLP1(
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
  (activation_function): ReLU()
)
MLP2(
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
  (activation_function): ReLU()
)
MLP3(
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=10, bias=True)
  (activation_function): ReLU()
)


# Definindo os Otimizadores e as Funções de Loss para cada Modelo

In [25]:

# Otimizadores para o Modelo de 1 camada oculta
opt_SGD_1     = torch.optim.SGD(model1.parameters(), lr=0.001, momentum=0.9)
opt_RMSprop_1 = torch.optim.RMSprop(model1.parameters(), lr=0.001, momentum=0.9)
opt_ADAM_1   = torch.optim.Adam(model1.parameters(), lr=0.001)

# Otimizadores para o Modelo de 2 camadas ocultas
opt_SGD_2     = torch.optim.SGD(model2.parameters(), lr=0.001, momentum=0.9)
opt_RMSprop_2 = torch.optim.RMSprop(model2.parameters(), lr=0.001, momentum=0.9)
opt_ADAM_2    = torch.optim.Adam(model2.parameters(), lr=0.001)

# Otimizadores para o Modelo de 3 camadas ocultas
opt_SGD_3     = torch.optim.SGD(model3.parameters(), lr=0.001, momentum=0.9)
opt_RMSprop_3 = torch.optim.RMSprop(model3.parameters(), lr=0.001, momentum=0.9)
opt_ADAM_3    = torch.optim.Adam(model3.parameters(), lr=0.001)

# Funções de Loss
loss_CE = nn.CrossEntropyLoss()
loss_MSE = nn.MSELoss()

# Definindo a função de Treinamento

In [23]:
# Função de Treinamento 
def train(dataset, model, optimizer, loss_fn, epochs=20):
    
    # Vetor de Loss Inicializado
    loss_array = []
    
    # Percore pelo número de épocas
    for epoch in range(epochs):
        
        # Inicializa o running_loss em 0
        running_loss = 0.0
        
        # Percorre todo o dataset
        for i, data in enumerate(dataset, 0):

            # Pega a Entrada e o Label
            input, label = data
            
            # Transforma para o device
            input, label = input.to(device), label.to(device)

            # Zera os parâmetros dos gradientes
            optimizer.zero_grad()
            
            # Passa as entradas pelo modelo
            output = model(input)
            
            # Calcula o loss com a função escolhida
            loss = loss_fn(output, label)
            
            # Propaga o erro
            loss.backward()
            
            # Atualiza os pesos
            optimizer.step()
            
            # Soma ao loss
            running_loss += loss.item()
        
        # Imprime o loss desta época
        loss_array.append(running_loss/len(dataset))
        print('[%3d] loss: %.4f' %
             (epoch + 1, loss_array[-1]))
    
    # Retorna o array com os Loss em cada época
    return loss_array


# Definindo a Função de Avalaliação do Dataset por um modelo especifico

In [24]:

# Função de avaliação do modelo num conjunto passado
def evaluate(dataset, model, type_of_images='test'):
    
    # Inicialização
    correct, total = 0, 0
    
    # Percorre todo o dataset
    with torch.no_grad():
        for data in dataset:
            
            # Transforma para o device
            image, label = data
            image, label = image.to(device), label.to(device)
            
            # Recebe a saida do modelo
            output = model(image)
            
            # Faz o resultado dos labels das saidas
            _, predicted = torch.max(output.data, 1)
            
            # Confere se o resultado foi igual o label para dar acerto
            total += 1 
            correct += (predicted == label).sum().item()
    
    # Imprime a acurácia do dataset para tal modelo
    print('Accuracy of the network on the %d %s images: %d %%' % 
          ( len(test_loader),  type_of_images, 100 * correct / total))
    
    # Retorna a acurácia
    return (100 * correct / total)


# __TREINAMENTO DOS MODELOS__

## Treinamento do Modelo de 1 Camada Oculta:

### Utilizando SGD + CrossEntropy

In [ ]:
SGD_1_time = time.time()
loss_SGD_CE_1 = train(train_loader, model1, opt_SGD_1, loss_CE)
SGD_1_time = time.time() - SGD_1_time

### Utilizando SGD + MSE

In [ ]:
loss_SGD_MSE_1 = train(train_loader, model1, opt_SGD_1, loss_MSE)

### Utilizado RMSprop + CrossEntropy

In [ ]:
loss_RMSprop_CE_1 = train(train_loader, model1, opt_RMSprop_1, loss_CE)

### Utilizando RMSprop + MSE

In [ ]:
loss_RMSprop_MSE_1 = train(train_loader, model1, opt_RMSprop_1, loss_MSE)

### Utilizando ADAM + CrossEntropy

In [ ]:
loss_ADAM_CE = train(train_loader, model1, opt_ADAM_1, loss_CE)

### Utilizando ADAM + MSE

In [ ]:
loss_ADAM_MSE = train(train_loader, model1, opt_ADAM_1, loss_MSE)

In [ ]:
SGD_2_time = time.time()
loss_SGD_CE_2 = train(train_loader, model2, opt_SGD_2, loss_CE)
SGD_2_time = time.time() - SGD_2_time

In [ ]:
SGD_3_time = time.time()
loss_SGD_CE_3 = train(train_loader, model3, opt_SGD_3, loss_CE)
SGD_3_time = time.time() - SGD_3_time

In [ ]:
print('tempo do modelo 1:', SGD_1_time)
print('tempo do modelo 2:', SGD_2_time)
print('tempo do modelo 3:', SGD_3_time)

### Gráfico do _CrossEntropyLoss_ utilizando _SGD_ no Modelo de 1 Camada Oculta

In [ ]:
plt.title('CrossEntropy + SGD no Modelo 1')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(loss_SGD_CE, 'b-')
plt.show()